## Mixed Integer Linear Programing 
### E-Scooter Sharing Scheme
Conditional Service 

In [ ]:
import pandas as pd
import numpy as np
import itertools
import os
import collections
import time
from pulp import *

In [ ]:
reduced = False
# 8, 12, 15
demand_percent = 12
# 0, 1, 2
# 0 - low (1 year)
# 1 - default (2.5 years)
# 2 - high (5 years)
costs = 1
budget = 1
timeLimit = 60*60*8

In [ ]:
problem = LpProblem("E-Scooter Allocation Demand Max", LpMaximize)

In [ ]:
if reduced:
    df_distance = pd.read_csv("Data/Preliminary analysis Data/Model Data - Smaller set Distance.csv")
else:
    df_distance = pd.read_csv("Data/Model Data - 56loc Distance.csv")

In [ ]:
df_distance.fillna(0, inplace=True)
distance = df_distance.values[:,1:]
distance_dict = { (x,y): distance[x][y] for x in range(distance.shape[0]) for y in range(distance.shape[1])}

In [ ]:
locations = list(df_distance['Location'])
location_idx = np.arange(0, len(locations))
loc_count = len(locations)

In [ ]:
if reduced:
    files = os.listdir('Data/Preliminary analysis Data/Reduced set Demand/')
else:
    files = os.listdir('Data/Final set Demand/')

In [ ]:
d = dict()
for f in files:
    hour = int(f.split(sep='.', maxsplit=1)[0])
    if reduced:
        df = pd.read_csv('Data/Preliminary analysis Data/Reduced set Demand/' + f)
    else:
        df = pd.read_csv('Data/Final set Demand/' + f)
    demand_arr = df.to_numpy()[:,1:]
    demand_arr = demand_arr*(demand_percent/15)
    for i in range(loc_count):
        for j in range(loc_count):
            demand_arr[i][j] = np.round(demand_arr[i][j],0)
    d[hour] = demand_arr

In [ ]:
demand = collections.OrderedDict(sorted(d.items())) 

In [ ]:
demand_dict = dict()
for i,(_,v) in enumerate(demand.items()):
    demand_dict.update({(x,y,i): v[x][y] for x in range(loc_count) for y in range(loc_count)})

In [ ]:
total_demand = 0
for _,v in demand_dict.items():
    total_demand+=v

In [ ]:
# Parameters
M = sys.maxsize
Zmax = 100
Zmin = 1
if costs == 0:
    # 1 year
    Cc_fixed_scooter = 140
    Cc_fixed_dock = 74
    Cc_fixed_station = 95

    Cp_fixed_scooter = 0.96
    Cp_fixed_dock = 2.3
    Cp_fixed_station = 5.75
elif costs == 1:
    # 2.5 years
    Cc_fixed_scooter = 55
    Cc_fixed_dock = 30
    Cc_fixed_station = 38

    Cp_fixed_scooter = 0.38
    Cp_fixed_dock = 0.92
    Cp_fixed_station = 2.3  
else:
    # 5 years
    Cc_fixed_scooter = 31
    Cc_fixed_dock = 15
    Cc_fixed_station = 19

    Cp_fixed_scooter = 0.19
    Cp_fixed_dock = 0.46
    Cp_fixed_station = 1.15

# Carbon Cost values
Cc_scooter_km = 7
# Cost relocation
Cc_r = 120

scalar = 1.25

# Cost values
# cost of maintaining one scooter per kilometer driven
Cp_scooter_km = 0.41

# Cost relocation
Cp_r = 1.5
# Price rate for kilometer driven
P_km = 0.6
# Price for pickup
P_init = 1
if reduced:
    B = 500
else:
    if budget == 0:
        B = 900
    elif budget == 1:
        B = 5000
    else:
        B = 1500

In [ ]:
# Penalty
if reduced:
    df_penalty = pd.read_csv("Data/Preliminary analysis Data/Penalty Carbon Costs Smaller set.csv")
else:
    df_penalty = pd.read_csv("Data/Penalty Carbon Costs 56loc.csv")
C_pen = df_penalty.values[:, 1]
# C_pen = C_pen*scalar

In [ ]:
# Sets
T = np.arange(0,len(files)+1)
# each location at a given time
X = list(itertools.product(location_idx, T))
X2 = [(i,it) for (i,it) in X if sum([demand_dict[(i,j,it)] for j in location_idx if i !=j and it != T[-1]]) > 0]
A1 = [(xi, xj) for xi in X for xj in X if xi[0] != xj[0] and xi[1]+1==xj[1]]
# Relocation
A2 = [(xi, xj) for xi in X for xj in X if xi[0]!=xj[0] and xi[1]==T[-1] and xj[1]==T[0]]

In [ ]:
# Decision Variables
Yi = LpVariable.dicts("Station Presence", location_idx,0,cat=const.LpBinary)
Zi = LpVariable.dicts("Size", location_idx, 0, cat=const.LpInteger)
# Relocation
Rij = LpVariable.dicts("#Relocated_Scooters", A2, 0, cat=const.LpInteger)
R = LpVariable.dicts('Relocation needed', A2, 0, cat=const.LpBinary)
Vit = LpVariable.dicts("#Available_Scooters",X,0,cat=const.LpInteger)
Ditj = LpVariable.dicts("#Used_Scooters", A1, 0 ,cat=const.LpInteger)
Xit = LpVariable.dicts("Availability binary", X2, 0, cat=const.LpBinary)

In [ ]:
# Objective function
problem+=lpSum(Ditj[i] for i in A1), "Objective function"

In [ ]:
for (i,ti) in X:
    if ti !=0:
        problem+= Vit[(i,ti-1)] - lpSum(Ditj[((i,ti-1),(j,ti))] for j in location_idx if i !=j) + lpSum(Ditj[((j, ti-1),(i, ti))] for j in location_idx if i !=j) == Vit[(i,ti)], f"Availability Balance {(i,ti)}"


In [ ]:
for (i,ti) in X:
    if ti!=T[-1]:
        problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,ti+1))] for j in location_idx if i !=j) >=0, f"Stocked Scooters for {(i,ti)}"

In [ ]:
# Relocation constraint
for i in location_idx:
    problem+=Vit[(i,0)] == Vit[(i,T[-1])] + lpSum(Rij[((j,T[-1]), (i,0))] for j in location_idx if i!=j) - lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j), f"Relocation balance for location {i}"

In [ ]:
for a2 in A2:
    problem+=Rij[a2]<=M*R[a2], f"Relocation needed for {a2}"

In [ ]:
for i in location_idx:
    problem+=lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j) <= Vit[(i,T[-1])], f"Relocation availability for location {i}"

In [ ]:
for (i,ti) in X:
    problem+= Zi[i] >=Vit[(i,ti)], f"Size constraint for {(i,ti)}"

In [ ]:
for (i,ti) in X:
    problem+=Vit[(i,ti)] <= Zmax*Yi[i], f"Maximum size constraint {(i,ti)}"

In [ ]:
for i in location_idx:
    problem+=Yi[i]*Zmin <=Zi[i], f"Minimum size constraint for {i}"

In [ ]:
for ((i,ti),(j,tj)) in A1:
    problem+=Ditj[((i,ti),(j,tj))]<=demand_dict[(i,j,ti)], f"Maximum Demand for {((i,ti),(j,tj))}"

In [ ]:
problem+=lpSum(Vit[(i,0)]*Cp_fixed_scooter for i in location_idx) + lpSum(Zi[i]*Cp_fixed_dock for i in location_idx) + lpSum(Yi[i]*Cp_fixed_station for i in location_idx)<=B, "Maximum Budget"

In [ ]:
for (i,ti) in X2:
    problem+= Xit[(i,ti)] <= (Vit[(i,ti)] + lpSum(demand_dict[(i,j,ti)] for j in location_idx if i!=j) - lpSum(demand_dict[(i,j,ti)]*Yi[j] for j in location_idx if i!=j))/lpSum(demand_dict[(i,j,ti)] for j in location_idx if i!=j), f"Xit value constraint for {(i,ti)}"

In [ ]:
for (i,ti) in X2:
    if ti != T[-1]:
        problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,ti+1))] for j in location_idx if i !=j) <= M*Xit[(i,ti)]

In [ ]:
for (i,ti) in X2:
    if ti != T[-1]:
        problem+= lpSum(Ditj[((i,ti), (j,ti+1))] for j in location_idx if i!=j) >= M*(Xit[(i,ti)]-1) + lpSum(demand_dict[(i,j,ti)]*Yi[j] for j in location_idx if i!=j)

In [ ]:
if reduced:
    logf = 'stats_demand_reduced.log'
else:
    logf = 'stats_demand.log'

In [ ]:
solver = apis.PULP_CBC_CMD(logPath=logf, options=['DivingVectorlength on', 'DivingSome on', '-cuts off'], gapRel=0.01, timeLimit = timeLimit)
start = time.time()
solver.actualSolve(problem)
end = time.time()
print(f'Running time: {str(int(end-start))}s')
LpStatus[problem.status]

In [ ]:
obj_val_carbon = sum([Ditj[((i,ti),(j,tj))].value()*distance_dict[(i,j)]*Cc_scooter_km for ((i,ti),(j,tj)) in A1]) + \
    sum([(demand_dict[(i,j,ti)]-Ditj[((i,ti),(j,tj))].value())*distance_dict[(i,j)]*C_pen[i] for ((i,ti),(j,tj)) in A1]) + \
    sum([R[((i,ti), (j,tj))].value()*distance_dict[(i,j)]*Cc_r for ((i,ti), (j,tj)) in A2]) + \
    sum([Vit[(i,t)].value()*Cc_fixed_scooter for (i,t) in X if t==0]) + \
    sum([Zi[i].value()*Cc_fixed_dock for i in location_idx]) + \
    sum([Yi[i].value()*Cc_fixed_station for i in location_idx])

In [ ]:
obj_val_profit = sum([Ditj[((i,ti),(j,tj))].value()*(distance_dict[(i,j)]*(P_km-Cp_scooter_km) + P_init) for ((i,ti),(j,tj)) in A1]) - \
    sum([R[((i,ti), (j,tj))].value()*distance_dict[(i,j)]*Cp_r for ((i,ti), (j,tj)) in A2]) - \
    sum([Vit[(i,t)].value()*Cp_fixed_scooter for (i,t) in X if t==0]) - \
    sum([Zi[i].value()*Cp_fixed_dock for i in location_idx]) - \
    sum([Yi[i].value()*Cp_fixed_station for i in location_idx])

In [ ]:
# Extracting the results
#(idx, LpVariable)
size_list = []
for _,v in Zi.items():
    size_list.append(v.value())  
docks_num = sum(size_list)

In [ ]:
station_list = []
for _,v in Yi.items():
    station_list.append(v.value())
stations_num = sum(station_list)

In [ ]:
df_results = pd.DataFrame(list(zip(locations, station_list, size_list)), columns=["Location","Station", "Size"])
df_results

In [ ]:
if reduced:
    df_results.to_csv("results-demand-reduced-NOcuts.csv", index=False)
else:
    df_results.to_csv("results-demand-NOcuts.csv", index=False)

In [ ]:
total_Vit = dict()
for k,v in Vit.items():
    if k[1] not in total_Vit:
        total_Vit[k[1]] = int(v.varValue)
    else:
        total_Vit[k[1]]+=int(v.varValue)
total_scooters = total_Vit[0]

In [ ]:
satisfied_demand = 0
for _,v in Ditj.items():
    satisfied_demand+=v.varValue

In [ ]:
relocated_scooters = 0
for k,v in Rij.items():
    if v.varValue>0:
        print(f"Rij[{k}] = {v.varValue}")
        relocated_scooters+=v.varValue

In [ ]:
fixed_cost = sum([Vit[(i,0)].value()*Cp_fixed_scooter for i in location_idx]) + sum([Zi[i].value()*Cp_fixed_dock for i in location_idx]) + sum([Yi[i].value()*Cp_fixed_station for i in location_idx])

In [ ]:
loc_trips = dict()

for l in location_idx:
    loc = locations[l]
    df = pd.DataFrame(columns=['Time', 'Available', 'Leaving', 'Arriving', 'Demand', 'Xit'])
    for t in T:
        if t!=T[-1]:
            available = Vit[(l,t)].value()
            leaving = sum([Ditj[((l,t),(j,t+1))].value() for j in location_idx if j!=l and t!=T[-1]])
            arriving = sum([Ditj[((j,t),(l,t+1))].value() for j in location_idx if j!=l and t!=T[-1]])
            time = f"{int(t+5)}:00-{int(t+5)}:59"
            d = sum([demand_dict[(l,j,t)] for j in location_idx if j!=l])
            x = Xit[(l,t)].value() if (l,t) in X2 else None
            df.loc[-1] = [time, int(available), int(leaving), int(arriving), int(d), x]
            df.index = df.index + 1
    relocate_from = sum([Rij[((l,T[-1]), (j,0))].value() for j in location_idx if j!=l])
    relocate_to = sum([Rij[((j,T[-1]), (l,0))].value() for j in location_idx if j!=l])
    df.loc[-1] = ['Relocate', None, int(relocate_from), int(relocate_to), None, None]
    df.index = df.index + 1
    loc_trips[loc] = df

In [ ]:
import report_generator as r
r.station_stats(loc_trips)

In [ ]:
import csv
variable_values = dict()
for v in problem.variables():
    variable_values[v.name] = v.value()

variable_values_df = pd.DataFrame.from_dict(variable_values, orient='index')
df_csv = variable_values_df.to_csv(index=True)

if reduced:
    filename = 'variable_values_demand_reduced_NOcuts.csv'
else:
    filename = 'variable_values_demand_NOcuts.csv'
 
with open(filename, 'w') as f:
    f.write(df_csv)

In [ ]:
print('------------- Statistics -------------')
print(f"Objective value carbon: {obj_val_carbon}")
print(f"Objective value profit: {obj_val_profit}")
print(f"Number of Scooters: {total_scooters}")
print(f'Trips made: {satisfied_demand}')
print(f'Satisfied demand: {satisfied_demand/total_demand}')
print(f'Avg. Scooter rides per day: {satisfied_demand/total_scooters}')
print(f'Number of Relocated scooters: {relocated_scooters}')
print(f"Total fixed cost: {fixed_cost}")
print(f'Number of docks: {docks_num}')
print(f'Number of stations: {stations_num}')